In [1]:
## VICTOR DICE:
import os
import gc
import time
import requests
import subprocess
import random as rn
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

import socks
import socket
import pymssql
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")
# warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

SEED = int(time.time() / 1000)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

In [4]:
agents_tor = [
    'Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gecko/20100101 Firefox/78.0'
]

headers_tor = {
    'content-type': "application/x-www-form-urlencoded",
    'User-Agent': rn.choice(agents_tor),
}

letras = "abcdefghijklmnopqrstuvwyz"

def traducir(texto, desde='es', hacia='en'):
    time.sleep(10)
    user_aletorio = ''.join([rn.choice(letras) for _ in range(8)])
    pass_aleatorio = ''.join([rn.choice(letras) for _ in range(8)])
    
    socks.set_default_proxy(socks.SOCKS5, "127.0.0.1", 9150, True, user_aletorio, pass_aleatorio)
    socket.socket = socks.socksocket
    
    headers_tor.update(
        {
            'User-Agent': rn.choice(agents_tor),
            'X-Forwarded-For': requests.get('http://ifconfig.me/ip').text
        }
    )
    
    url_ = 'https://translate.googleapis.com/translate_a/single?client=gtx&sl={}&tl={}&dt=t&q={}'
    response = requests.get(
        url_.format(desde, hacia, texto), headers=headers_tor
    )
    
    return response.json()[0][0][0] if response.status_code == 200 else 'ERROR'

In [5]:
traducir('🚨 es importante que sepas esto')

'🚨 it is important that you know this'

### DATA

In [6]:
data = pd.read_csv('campanias.csv')
data.head()

,ndeg,id-envio,fecha,hora,feedback-hasta,empresa,usuario,nombre,boletin,cod-archivo,asunto,cantidad-carga,cantidad-enviado,entregado,cola-de-envio,ignorado,rebotado-soft,rebotado,fallidos,open-rate,click
0,119132,15620,2021-03-09,14:30,2021-03-10 20:56:15,Digital Transformation Executive,pvalero,(final) 09032021_6437_Envio HTML Interbank 09/03_comercio,09032021_6437_Envio HTML Interbank 09/03_comercio,09032021_6437_Envio HTML Interbank 09/03_comercio,llega a mas clientes vendiendo por internet 💻👌,18695,18090,14924,0,2809,334,23,605,1062,44
1,119131,15619,2021-03-09,02:30,2021-03-10 20:59:30,Digital Transformation Executive,pvalero,(final) 09032021_6436_Envio HTML Interbank 09/03_servicios,09032021_6436_Envio HTML Interbank 09/03_servicios,09032021_6436_Envio HTML Interbank 09/03_servicios,incrementa tu productividad y ahorra costos en tu negocio 💻👌,38163,36696,32320,0,3432,893,51,1467,2660,94
2,119053,15599,2021-03-03,17:51,2021-03-05 00:55:07,Digital Transformation Executive,pvalero,(final) 03032021_6382_HTML - Envío Canal Seguro BE,03032021_6382_HTML - Envío Canal Seguro BE,03032021_6382_HTML - Envio Canal Seguro BE,Conoce las medidas de seguridad de App Interbank Empresas,2137,2122,1339,0,459,290,34,15,400,43
3,118972,15564,2021-02-25,13:41,2021-03-03 01:10:14,Digital Transformation Executive,pvalero,(final) 25022021_6318_HTML televentas grupo 2,25022021_6318_HTML televentas grupo 2,25022021_6318_HTML televentas grupo 2,conoce el tipo de cambio exclusivo para tu empresa,2127,2091,1804,0,174,113,0,36,546,11
4,118955,15555,2021-02-24,12:56,2021-03-01 22:23:10,Digital Transformation Executive,pvalero,(final) 24022021_6301_html Interbank - Envio 24/02,24022021_6301_html Interbank - Envio 24/02,24022021_6301_html Interbank - Envio 2402,juntos impulsaremos tu negocio gratis y desde casa.,21082,20451,18299,0,1567,567,18,631,2653,466


In [7]:
data['asunto_ingles'] = data['asunto'].apply(lambda _: traducir(_))

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
data['asunto_ingles'] = data['asunto_ingles'].apply(lambda _: traducir(_) if 'ERROR' in _.upper() else _)

In [ ]:
traducir(
    list(data[data['asunto_ingles'] == 'ERROR']['asunto'])[0]
)

In [ ]:
data.loc[37, 'asunto_ingles'] = 'Do you already know the special exchange rate we have for your company?'

In [ ]:
data[['asunto_ingles', 'asunto']].tail(60)

In [ ]:
data.dtypes

In [ ]:
data.isnull().sum()

### Conversion de emojis y/o emoticones ---> significado

In [ ]:
import emot

text = ':-) Woohooo😍✌️'

def clean_mean(val):
    return val.replace('_', ' ').replace('-', ' ').replace(':', ' ')

def handle_emoji_emoti(text):
    for emoti in emot.emo_unicode.EMOTICONS:
        if emoti in text:
            text = text.replace(emoti, ' ' + clean_mean(emot.emo_unicode.EMOTICONS.get(emoti, '')))
            print(emoti, end=' ')

    for emoti in emot.emo_unicode.UNICODE_EMO:
        if emoti in text:
            text = text.replace(emoti, ' ' + clean_mean(emot.emo_unicode.UNICODE_EMO.get(emoti, '')))
            print(emoti, end=' ')

    for emoti in emot.emo_unicode.EMOTICONS_EMO:
        if emoti in text:
            text = text.replace(emoti, ' ' + clean_mean(emot.emo_unicode.EMOTICONS_EMO.get(emoti, '')))
            print(emoti, end=' ')
        
    return text
        
text = handle_emoji_emoti(text)
print(text)

In [ ]:
data['asunto_ingles'] = data['asunto_ingles'].apply(lambda _: handle_emoji_emoti(_))

In [ ]:
data[['asunto_ingles', 'asunto']].head(60)

### RATIOS

In [ ]:
data.head()

In [ ]:
data['open_rate'] = data[['entregado', 'open-rate']].apply(lambda _: _[1] / _[0], axis=1)
data['open_rate'].hist()

In [ ]:
data['ratio_click_campania'] = data[['entregado', 'click']].apply(lambda _: _[1] / _[0], axis=1)
data['ratio_click_campania'].hist()

In [ ]:
data['open_rate_later_click'] = data[['open-rate', 'click']].apply(lambda _: _[1] / _[0], axis=1)
data['open_rate_later_click'].hist()

### SENTIMENT

In [ ]:
import nltk
from nltk.corpus import stopwords
stopwords_english = stopwords.words('english')

print(stopwords_english)

In [ ]:
data['asunto_ingles'] = data['asunto_ingles'].apply(
    lambda _:  ' '.join([word for word in _.split() if word not in stopwords_english]).strip()
)

### AGREGANDO MAYOR INFLUENCIA "!"

In [ ]:
def convert_upper_for_admiration(text_1):
    for word in text_1.split():
        if '!' in word:
            text_1 = text_1.replace(word, word.upper())
    
    return text_1

data['asunto_ingles'] = data['asunto_ingles'].apply(lambda _: convert_upper_for_admiration(_))

In [ ]:
data['asunto_ingles'].head(60)

### APPLY SENTUMENT WITH VADER

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [ ]:
data['sentiment_dict'] = data['asunto_ingles'].apply(lambda _: analyser.polarity_scores(_))
data['sentiment_dict'].head()

In [ ]:
data['negativo'] = data['sentiment_dict'].apply(lambda _: _.get('neg', 0))
data['positivo'] = data['sentiment_dict'].apply(lambda _: _.get('pos', 0))
data['neutro'] = data['sentiment_dict'].apply(lambda _: _.get('neu', 0))
data['composicion'] = data['sentiment_dict'].apply(lambda _: _.get('compound', 0))

In [ ]:
data['negativo'].hist()

In [ ]:
data['positivo'].hist()

In [ ]:
data['neutro'].hist()

In [ ]:
data['composicion'].hist()

### CORRELACION

In [ ]:
data[['open_rate', 'open_rate_later_click', 'ratio_click_campania', 'neutro', 'positivo', 'negativo', 'composicion']].corr(
    method='spearman').style.background_gradient(cmap='coolwarm', axis=None).set_precision(3)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

g = sns.PairGrid(data, vars=["negativo","neutro", "composicion", "positivo"])
g.map_diag(sns.kdeplot)
g.map_offdiag(sns.scatterplot, hue=data["open_rate"], s=100, linewidth=0, palette='YlOrBr')
g.axes[1, -1].legend(loc="center left", bbox_to_anchor=(.7, .5))

In [ ]:
def graph_relation_continuos(data, col_x, col_y):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

    sns.kdeplot(
        data=data, x=col_x, y=col_y, fill=True, ax=axes[0]
    )
    sns.regplot(
        x=col_x, y=col_y, data=data, robust=True, ax=axes[1]
    )

In [ ]:
graph_relation_continuos(data, "neutro", "open_rate")    ---- open rate

In [ ]:
graph_relation_continuos(data, "neutro", "ratio_click_campania")

In [ ]:
graph_relation_continuos(data, "positivo", "open_rate")

In [ ]:
graph_relation_continuos(data, "positivo", "ratio_click_campania")

In [ ]:
graph_relation_continuos(data, "negativo", "open_rate")

In [ ]:
data[data['negativo'] > 0.3][['asunto', 'asunto_ingles', 'open_rate', 'neutro', 'positivo', 'negativo', 'composicion']].head(60)

In [ ]:
data[data['open_rate'] > 0.3][['asunto', 'asunto_ingles', 'open_rate', 'neutro', 'positivo', 'negativo', 'composicion']]

In [ ]:
graph_relation_continuos(data, "composicion", "open_rate")

In [ ]:
sns.kdeplot(
    data=data, x='positivo', y='neutro', fill=True
)

In [ ]:
123024 / 3

In [ ]:
len("""
sambraonline.com The DNS query name does not exist: sambraonline.com.                          
""")

In [ ]:
len("""
sambraonline.com The DNS query name does not exist: sambraonline.com.                  
""")